## **🧠 Understanding Nested Subqueries in SQL**
### 📌 What is a Subquery?
- A subquery (also called an inner query or nested query) is a query embedded within another SQL query. It is used to compute intermediate results that the outer query depends on.

---

#### 🔹 1. Set Membership (IN clause)
- Used to test whether a value is a member of a set of values returned by a subquery.
    ```sql
    SELECT name
    FROM student
    WHERE dept_id IN (
        SELECT dept_id FROM department WHERE location = 'Delhi'
    );
    ```

---

#### 🔹 2. Set Comparison – SOME Clause
- 📘 Definition:
    - The SOME operator (also known as ANY) compares a value to some or at least one value returned by a subquery.

    ```sql
    SELECT name
    FROM employee
    WHERE salary > SOME (
        SELECT salary FROM employee WHERE dept_id = 10
    );```

- This returns employees whose salary is greater than at least one salary in dept 10.

---

#### 🔹 3. Set Comparison – ALL Clause
- 📘 Definition:
    - The ALL operator compares a value to all values returned by the subquery.

    ```sql
    SELECT name
    FROM employee
    WHERE salary > ALL (
        SELECT salary FROM employee WHERE dept_id = 20
    );```
- Returns employees whose salary is greater than every salary in dept 20.

---

#### 4. Test for Empty Relations – EXISTS
- ✅ Use of EXISTS:
    - Returns TRUE if the subquery returns at least one row.

    ```sql
    SELECT name
    FROM student s
    WHERE EXISTS (
        SELECT * FROM enrollment e WHERE e.student_id = s.id
    );
    ```

---

#### 🔹 5. Use of NOT EXISTS
- Returns TRUE if the subquery returns no rows.

    ```sql
    SELECT name
    FROM student s
    WHERE NOT EXISTS (
        SELECT * FROM enrollment e WHERE e.student_id = s.id
    );
    ```

- Finds students not enrolled in any course.

---

#### 🔹 6. Test for Absence of Duplicate Tuples – UNIQUE
- The UNIQUE keyword is used to ensure the subquery does not return duplicate rows.

    ```sql
    SELECT name
    FROM student
    WHERE UNIQUE (
        SELECT course_id FROM enrollment WHERE student_id = student.id
    );
    ```

- Returns students enrolled in at most one unique course (rarely used in practice; use GROUP BY or constraints instead).

--- 

#### 🔹 7. Subqueries in the FROM Clause
- Subqueries can act like temporary tables when placed in the FROM clause.

    ```sql
    SELECT avg_salary
    FROM (
        SELECT dept_id, AVG(salary) AS avg_salary
        FROM employee
        GROUP BY dept_id
    ) AS dept_avg
    WHERE dept_id = 101;
    ```

---

#### 🔹 8. WITH Clause (Common Table Expressions - CTE)
- Used to define temporary result sets that can be reused in the query.

    ```sql
    WITH dept_avg AS (
        SELECT dept_id, AVG(salary) AS avg_sal
        FROM employee
        GROUP BY dept_id
    )
    SELECT e.name
    FROM employee e
    JOIN dept_avg d ON e.dept_id = d.dept_id
    WHERE e.salary > d.avg_sal;

---

#### 🔹 9. Complex Queries using WITH Clause

```sql
    WITH high_earners AS (
        SELECT emp_id, salary
        FROM employee
        WHERE salary > 8000
    ),
    bonus_eligible AS (
        SELECT emp_id
        FROM performance
        WHERE rating = 'A'
    )
    SELECT name
    FROM employee
    WHERE emp_id IN (
        SELECT emp_id FROM high_earners
        INTERSECT
        SELECT emp_id FROM bonus_eligible
    );
```

---

#### 🔹 10. Subqueries in the SELECT Clause (Scalar Subquery)
- A scalar subquery returns a single value used directly in the select list.

```sql
    SELECT name,
        (SELECT COUNT(*) FROM enrollment e WHERE e.student_id = s.id) AS total_courses
    FROM student s;
```

---

###  🛠️ Modifications of the Database

##### 🔸 Deletion

```sql
    DELETE FROM student
    WHERE id = 101;
```
    
- With subquery:

```sql
    DELETE FROM student
    WHERE id IN (SELECT student_id FROM dropout_list);
```

---

##### 🔸 Insertion
    
```sql
    INSERT INTO alumni (id, name)
    SELECT id, name FROM student WHERE graduated = TRUE;
```

---

##### 🔸 Updates

```sql
    UPDATE student
    SET age = 22
    WHERE id = 105;
```
- With subquery:

```sql
    UPDATE student
    SET dept_id = (
        SELECT dept_id FROM department WHERE dept_name = 'Science'
    )
    WHERE name = 'John';
```

---

### 🔁 CASE Statement for Conditional Updates

```sql
    UPDATE employee
    SET bonus = 
        CASE
            WHEN salary > 10000 THEN 2000
            WHEN salary BETWEEN 8000 AND 10000 THEN 1000
            ELSE 500
        END;
```

---

### 🔁 Updates with Scalar Subqueries

```sql
    UPDATE employee e
    SET salary = salary + (
        SELECT bonus_amount FROM bonus_plan b WHERE b.grade = e.grade
    )
    WHERE EXISTS (
        SELECT * FROM bonus_plan b WHERE b.grade = e.grade
    );
```


